<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/cifar10_cnn_resnet%2Bsenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import numpy as np
from keras.datasets import cifar10
from functools import partial
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape, x_test.shape
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
y_train = y_train.astype(np.int32).reshape(-1)
y_test = y_test.astype(np.int32).reshape(-1)
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(x_train, test_size=0.2, random_state=42)
y_train, y_val = train_test_split(y_train, test_size=0.2, random_state=42)

Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step


In [3]:
import time
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

def shuffle_batch(x, y, batch_size):
  rnd_idx = np.random.permutation(len(x))
  n_batch = len(x) // batch_size
  for batch_idx in np.array_split(rnd_idx, n_batch):
    xs, ys = x[batch_idx], y[batch_idx]
    yield xs, ys

name = ['drop', 'layer', 'bn', 'act']
name_list = []
for i in range(4):
  bb = [b + str(i) for b in name]
  name_list.append(bb)
  bb = []

n_layers=4
n_epochs = 101
batch_size = 1000
x = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name="x")
y = tf.placeholder(dtype=tf.int32, shape=[None], name="y")
training = tf.placeholder_with_default(False, shape=[], name="training")
he_init = tf.variance_scaling_initializer()

##########partial########
conv = partial(tf.layers.conv2d, kernel_size=3, padding='SAME', kernel_initializer=he_init)
conv_re = partial(tf.layers.conv2d, kernel_size=1, strides=2, padding='SAME', kernel_initializer=he_init)
max_pool = partial(tf.layers.max_pooling2d, padding='SAME', strides=2)
avg_pool = partial(tf.layers.average_pooling2d, padding='VALID', strides=1)
bn = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
drop = partial(tf.layers.dropout, rate=0.5, training=training)

##########convolution########
conv1 = conv(x, filters=64, strides=1, name='conv1')
bn1 = bn(conv1, name='bn1')
act1 = tf.nn.relu(bn1)
max_pool1 = max_pool(act1, pool_size=3, name='max_pool1')

conv2 = conv(max_pool1, filters=64, strides=1, name='conv2')
bn2 = bn(conv2, name='bn2')
act2 = tf.nn.relu(bn2, name='act2')
conv2_2 = conv(act2, filters=64, strides=1, name='conv2_2')
bn2_2 = bn(conv2_2, name='bn2_2')
##se block
avg_pool2 = avg_pool(bn2_2, pool_size=16, name='avg_pool2')
se_block2_1 = tf.layers.dense(avg_pool2, 64/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block2_1')
se_block2_2 = tf.layers.dense(se_block2_1, 64, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block2_2')
res2 = tf.add(max_pool1, bn2_2*se_block2_2, name='res2')
act2_2 = tf.nn.relu(res2, name='act2_2')
##

conv3 = conv(act2_2, filters=64, strides=1, name='conv3')
bn3 = bn(conv3, name='bn3')
act3 = tf.nn.relu(bn3, name='act3')
conv3_2 = conv(act3, filters=64, strides=1, name='conv3_2')
bn3_2 = bn(conv3_2, name='bn3_2')
##se block
avg_pool3 = avg_pool(bn3_2, pool_size=16, name='avg_pool3')
se_block3_1 = tf.layers.dense(avg_pool3, 64/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block3_1')
se_block3_2 = tf.layers.dense(se_block3_1, 64, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block3_2')
res3 = tf.add(act2_2, bn3_2*se_block3_2, name='res3')
act3_2 = tf.nn.relu(res3, name='act3_2')
#######

conv4 = conv(act3_2, filters=128, strides=2, name='conv4')
bn4 = bn(conv4, name='bn4')
act4 = tf.nn.relu(bn4, name='act4')
conv4_2 = conv(act4, filters=128, strides=1, name='conv4_2')
bn4_2 = bn(conv4_2, name='bn4_2')
avg_pool4 = avg_pool(bn4_2, pool_size=8, name='avg_pool4')
se_block4_1 = tf.layers.dense(avg_pool4, 128/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block4_1')
se_block4_2 = tf.layers.dense(se_block4_1, 128, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block4_2')
conv4_re = conv_re(act3_2, filters=128, name='conv4_re')
bn4_re = bn(conv4_re, name='bn4_re')
res4 = tf.add(bn4_re, bn4_2*se_block4_2, name='res4')
act4_2 = tf.nn.relu(res4, name='act4_2')
##
conv5 = conv(act4_2, filters=128, strides=1, name='conv5')
bn5 = bn(conv5, name='bn5')
act5 = tf.nn.relu(bn5, name='act5')
conv5_2 = conv(act5, filters=128, strides=1, name='conv5_2')
bn5_2 = bn(conv5_2, name='bn5_2')
avg_pool5 = avg_pool(bn5_2, pool_size=8, name='avg_pool5')
se_block5_1 = tf.layers.dense(avg_pool5, 128/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block5_1')
se_block5_2 = tf.layers.dense(se_block5_1, 128, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block5_2')
res5 = tf.add(act4_2, bn5_2*se_block5_2, name='res5')
act5_2 = tf.nn.relu(res5, name='act5_2')
#######

conv6 = conv(act5_2, filters=256, strides=2, name='conv6')
bn6 = bn(conv6, name='bn6')
act6 = tf.nn.relu(bn6, name='act6')
conv6_2 = conv(act6, filters=256, strides=1, name='conv6_2')
bn6_2 = bn(conv6_2, name='bn6_2')
avg_pool6 = avg_pool(bn6_2, pool_size=4, name='avg_pool5')
se_block6_1 = tf.layers.dense(avg_pool6, 256/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block6_1')
se_block6_2 = tf.layers.dense(se_block6_1, 256, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block6_2')
conv6_re = conv_re(act5_2, filters=256, name='conv6_re')
bn6_re = bn(conv6_re, name='bn6_re')
res6 = tf.add(bn6_re, bn6_2*se_block6_2, name='res6')
act6_2 = tf.nn.relu(res6, name='act6_2')
##
conv7 = conv(act6_2, filters=256, strides=1, name='conv7')
bn7 = bn(conv7, name='bn7')
act7 = tf.nn.relu(bn7, name='act7')
conv7_2 = conv(act7, filters=256, strides=1, name='conv7_2')
bn7_2 = bn(conv7_2, name='bn7_2')
avg_pool7 = avg_pool(bn7_2, pool_size=4, name='avg_pool7')
se_block7_1 = tf.layers.dense(avg_pool7, 256/2, kernel_initializer=he_init, activation=tf.nn.relu, name='se_block7_1')
se_block7_2 = tf.layers.dense(se_block7_1, 256, kernel_initializer=he_init, activation=tf.nn.sigmoid, name='se_block7_2')
res7 = tf.add(act6_2, bn7_2*se_block7_2, name='res7')
act7_2 = tf.nn.relu(res7, name='act7_2')

###############

avg_pool = tf.layers.average_pooling2d(act7_2, pool_size=4, strides=(1,1), padding='VALID', name='avg_pool')
reshape = tf.reshape(avg_pool, shape=[-1, 256])

####fully connected#####
drop5 = tf.layers.dropout(reshape, rate=0.5, training=training, name="drop5")
layer5 = tf.layers.dense(drop5, 10, kernel_initializer=he_init, name="layer5")
################################

logits = layer5
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="xentropy")
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(0.001, name="optimizer")
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
cost_sum = []
file_writer = tf.summary.FileWriter("./", tf.get_default_graph())
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for xs, ys in shuffle_batch(x_train, y_train, batch_size):
      _, cost, __ = sess.run([training_op, loss, extra_update_ops], feed_dict={x: xs, y: ys, training: True})
      cost_sum.append(cost)
    cost_avg = sum(cost_sum) / len(cost_sum)
    if epoch % 10 == 0:
      acc_val = sess.run(accuracy, feed_dict={x: x_val, y: y_val})
      print("{}번째\t손실: {}\t테스트정확도: {}".format(epoch, cost_avg, acc_val))
    cost_avg = 0.0
  save_path = saver.save(sess, "./model")
  print("걸린시간: ", time.time()-start)
# with tf.Session() as sess:
#   init.run()
#   a = conv3.eval(feed_dict={x: x_test, y: y_test})
#   print(a.shape)



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
0번째	손실: 1.5128108203411101	테스트정확도: 0.43299999833106995
10번째	손실: 0.5134766824035482	테스트정확도: 0.7060999870300293
20번째	손실: 0.29100308837147343	테스트정확도: 0.7533000111579895
30번째	손실: 0.206205931843649	테스트정확도: 0.7506999969482422
40번째	손실: 0.16108229042076302	테스트정확도: 0.7646999955177307
50번째	손실: 0.1332953241120741	테스트정확도: 0.7634999752044678
60번째	손실: 0.11398837121716915	테스트정확도: 0.7631000280380

In [4]:
with tf.Session() as sess:
  saver.restore(sess, "./model")
  acc_test = accuracy.eval(feed_dict={x: x_test, y: y_test})
  print(acc_test)

INFO:tensorflow:Restoring parameters from ./model
0.7939
